In [16]:
# les bibliotheques nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import shap
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore')
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, StackingRegressor
from sklearn.model_selection import cross_validate, KFold
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [17]:
print("\n1. CHARGEMENT DES DONNÉES")
df_engineered = pd.read_csv("donnees_ameliorees.csv")
print(f"Shape des données : {df_engineered.shape}")
df_engineered.columns


1. CHARGEMENT DES DONNÉES
Shape des données : (669, 21)


Index(['PropertyGFATotal', 'PropertyGFABuilding(s)', 'NumberofFloors',
       'NumberofBuildings', 'YearBuilt', 'PrimaryPropertyType',
       'LargestPropertyUseTypeGFA', 'ZipCode', 'Neighborhood', 'Latitude',
       'Longitude', 'ENERGYSTARScore', 'log_TotalGHGEmissions',
       'log_SiteEnergyUse_kBtu', 'BuildingAge', 'AvgFloorArea',
       'GFATotal_per_Building', 'PrimaryPropertyType_grouped',
       'log_PropertyGFATotal', 'log_LargestPropertyUseTypeGFA',
       'log_AvgFloorArea'],
      dtype='object')

In [18]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

## DÉFINITION DES FEATURES ET DE LA CIBLE
# On définit la nouvelle cible
target = 'log_SiteEnergyUse_kBtu'

# Listes de features 
numerical_features = [
    'log_PropertyGFATotal', 'log_LargestPropertyUseTypeGFA', 'log_AvgFloorArea',
    'BuildingAge', 'NumberofFloors', 'NumberofBuildings', 'Latitude', 'Longitude'
]
categorical_features = ['PrimaryPropertyType_grouped']

all_features = numerical_features + categorical_features

# On crée X et y
X = df_engineered[all_features]
y = df_engineered[target]

print("✓ Features et nouvelle cible sélectionnées.")

# DÉFINITION DU PREPROCESSEUR ET DIVISION STRATIFIÉE DES DONNÉES

#préprocesseur
preproc = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])


#  On crée des catégories (strates) à partir de la cible continue
y_strat = pd.qcut(y, q=4, labels=False, duplicates='drop')

# On utilise ces strates dans le train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y_strat # Ajout de la stratification
)

print(f"\n✓ Données prêtes pour l'entraînement (avec division stratifiée) :")
print(f"  - Taille de X_train : {X_train.shape}")
print(f"  - Taille de X_test : {X_test.shape}")

✓ Features et nouvelle cible sélectionnées.

✓ Données prêtes pour l'entraînement (avec division stratifiée) :
  - Taille de X_train : (535, 9)
  - Taille de X_test : (134, 9)


In [19]:

# Configuration de la validation croisée 
cv = KFold(n_splits=5, shuffle=True, random_state=42)
# 1. Créer les catégories (strates) à partir de la variable cible 'y'
#    Cette étape est indispensable pour la stratification en régression.
y_strat = pd.qcut(y, q=4, labels=False, duplicates='drop')

# 2. Définir l'objet de validation croisée stratifiée à utiliser dans les GridSearchCV

cv_strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Liste des métriques 
scoring = {
    'MAE':  'neg_mean_absolute_error',
    'RMSE': 'neg_root_mean_squared_error', 
    'R2':   'r2'
}

In [20]:
# mise en place du modèle de base DummyRegressor - prédit juste la moyenne
print("1. MODÈLE BASELINE (DummyRegressor)")
print("-"*50)

dummy_pipe = Pipeline([
    ('prep', preproc),
    ('model', DummyRegressor(strategy='mean'))
])

# Cross-validation du baseline
cv_dummy = cross_validate(dummy_pipe, X_train, y_train, cv=cv, scoring=scoring)

baseline_r2 = cv_dummy['test_R2'].mean()
baseline_rmse = -cv_dummy['test_RMSE'].mean()
baseline_mae = -cv_dummy['test_MAE'].mean()

print(f"Baseline (DummyRegressor - moyenne) :")
print(f"  R2 CV    : {baseline_r2:.4f} ± {cv_dummy['test_R2'].std():.4f}")
print(f"  RMSE CV  : {baseline_rmse:.4f} ± {cv_dummy['test_RMSE'].std():.4f}")
print(f"  MAE CV   : {baseline_mae:.4f} ± {cv_dummy['test_MAE'].std():.4f}")
print(f" on note ici que nos modèles à entrainer doivent avoir des résultats inférieurs au niveau des RMSE et MAE à ces moyennes de base") 
print(f" pour le coefficent de détermination (R^2) l'objectif est d'avoir une valeur supérieure à - 0.0135, l'idéal étant un score positif")

1. MODÈLE BASELINE (DummyRegressor)
--------------------------------------------------
Baseline (DummyRegressor - moyenne) :
  R2 CV    : -0.0135 ± 0.0075
  RMSE CV  : 1.2054 ± 0.1155
  MAE CV   : 0.9522 ± 0.1024
 on note ici que nos modèles à entrainer doivent avoir des résultats inférieurs au niveau des RMSE et MAE à ces moyennes de base
 pour le coefficent de détermination (R^2) l'objectif est d'avoir une valeur supérieure à - 0.0135, l'idéal étant un score positif


In [21]:
print("\n2. RÉGRESSION LINÉAIRE SIMPLE")
print("-" * 50)

# La régression linéaire sert de baseline pour comprendre
# les relations linéaires dans nos données avant d'ajouter de la régularisation
linear_pipe = Pipeline([('prep', preproc), ('model', LinearRegression())])
cv_linear  = cross_validate(
    linear_pipe,
    X_train, y_train,
    cv=cv,
    scoring=scoring,
    return_train_score=True   # ← on récupère train_X et test_X
)
# très bon résultat et  point de départ, il explique 66 % de la variance, écart train/test limité (~0.04).
print(f"  R^2 train CV : {cv_linear ['train_R2'].mean():.4f} ± {cv_linear ['train_R2'].std():.4f}")
print(f"  R^2  test  CV : {cv_linear ['test_R2'].mean():.4f} ± {cv_linear ['test_R2'].std():.4f}")
print(f"  RMSE train CV : {-cv_linear ['train_RMSE'].mean():.4f} ± {cv_linear ['train_RMSE'].std():.4f}")
print(f"  RMSE test  CV : {-cv_linear ['test_RMSE'].mean():.4f} ± {cv_linear ['test_RMSE'].std():.4f}")
print(f"  MAE train CV  : {-cv_linear ['train_MAE'].mean():.4f} ± {cv_linear ['train_MAE'].std():.4f}")
print(f"  MAE test  CV  : {-cv_linear ['test_MAE'].mean():.4f} ± {cv_linear ['test_MAE'].std():.4f}")


2. RÉGRESSION LINÉAIRE SIMPLE
--------------------------------------------------


  R^2 train CV : 0.7072 ± 0.0122
  R^2  test  CV : 0.6603 ± 0.0330
  RMSE train CV : 0.6529 ± 0.0161
  RMSE test  CV : 0.6955 ± 0.0606
  MAE train CV  : 0.4923 ± 0.0117
  MAE test  CV  : 0.5292 ± 0.0447


In [22]:

print("3. MODÈLES RÉGULARISÉS (Ridge, Lasso, ElasticNet)")
print("-"*50)

#  La régularisation aide à :
# - Ridge : réduire l'overfitting en pénalisant les coefficients élevés
# - Lasso : sélection de variables en annulant certains coefficients
# - ElasticNet : combinaison des avantages de Ridge et Lasso
# modèles linéaires

# Pipeline de base
reg_pipe = Pipeline([
    ('prep', preproc),
    ('model', Ridge())  # Placeholder qui sera remplacé par GridSearchCV
])

# Grille de paramètres - pour tester plusieurs combinaisons d'hyperparamètres pour donner le meilleur compromis biais/variance 
#alpha = force de pénalité pour essayer de réduire la variance (moins de sur-entrainement), stabiliser les poids de chaque feature, et sélectionner des variables via Lasso si nécessaire
#Rideg pénalise la somme dess poids au carré, plus Alpha est grand plus les coeff sont retrécies vers 0, Lasso pénalise la somme des valeurs absolues et Elastic net combien les 2 (sélection et stabilisation)
param_grid = [
    {
        'model': [Ridge()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0] # échelle logarithmique pour balayer plusieurs ordres de grandeur 
        #et voir si le modèle préfère une pénalité nulle, modérée ou forte
    },
    {
        'model': [Lasso()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]  #idem
    },
    {
        'model': [ElasticNet()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
        'model__l1_ratio': [0.1, 0.5, 0.7, 0.9] # curseur de mélange pour "doser" Lasso et Ridge - exemple 0.5 c'est 50% de Lasso et 50% de Ridge , 0.9 surtout Lasso et peu de Ridge
    }
]

print("Optimisation des hyperparamètres par GridSearchCV...")
gs_reg = GridSearchCV(
    reg_pipe,
    param_grid,
    cv=cv,
    scoring=scoring,
    refit='RMSE', #moins standart que R^2 mais logique dans ma démarche
    n_jobs=-1,
    return_train_score=True  # important pour avoir mean_train_...
)
gs_reg.fit(X_train, y_train) #on fit le jeu de données

cv_res = gs_reg.cv_results_
best_idx = gs_reg.best_index_   # l’indice de la meilleure config trouvée

best_reg_r2   = cv_res['mean_test_R2'][best_idx]
best_reg_rmse = -cv_res['mean_test_RMSE'][best_idx]
best_reg_mae  = -cv_res['mean_test_MAE'][best_idx]

# On récupère les meilleurs index par type
best_ridge_idx = None
best_lasso_idx = None
best_enet_idx = None

for i, p in enumerate(cv_res['params']):
    if isinstance(p['model'], Ridge):
        if best_ridge_idx is None or cv_res['mean_test_R2'][i] > cv_res['mean_test_R2'][best_ridge_idx]:
            best_ridge_idx = i
    elif isinstance(p['model'], Lasso):
        if best_lasso_idx is None or cv_res['mean_test_R2'][i] > cv_res['mean_test_R2'][best_lasso_idx]:
            best_lasso_idx = i
    elif p['model'].__class__.__name__ == 'ElasticNet':
        if best_enet_idx is None or cv_res['mean_test_R2'][i] > cv_res['mean_test_R2'][best_enet_idx]:
            best_enet_idx = i

print("\n=== Résultats Régularisation ===")

for name, idx in [('Ridge', best_ridge_idx), ('Lasso', best_lasso_idx), ('ElasticNet', best_enet_idx)]:
    if idx is None:
        print(f"{name} : Non testé")
        continue
   
    print(f"\n{name} :")
    print(f"  Meilleurs paramètres : {cv_res['params'][idx]}")
    print(f"  R^2 train CV : {cv_res['mean_train_R2'][idx]:.4f}")
    print(f"  R^2 test  CV : {cv_res['mean_test_R2'][idx]:.4f}")
    print(f"  RMSE train CV : {-cv_res['mean_train_RMSE'][idx]:.4f}")
    print(f"  RMSE test  CV : {-cv_res['mean_test_RMSE'][idx]:.4f}")
    print(f"  MAE train CV  : {-cv_res['mean_train_MAE'][idx]:.4f}")
    print(f"  MAE test  CV  : {-cv_res['mean_test_MAE'][idx]:.4f}")
# ANALYSE :
#les 3 modèles sont très bien régularisés, les variances sont faibles
# Le modèle Ridge se distingue comme le plus performant. Il obtient le meilleur score R² en validation croisée (0.6603) et 
# l'erreur RMSE la plus faible (0.6955).
#  Cela signifie qu'il est le plus précis des trois pour prédire notre cible.
   

3. MODÈLES RÉGULARISÉS (Ridge, Lasso, ElasticNet)
--------------------------------------------------
Optimisation des hyperparamètres par GridSearchCV...

=== Résultats Régularisation ===

Ridge :
  Meilleurs paramètres : {'model': Ridge(), 'model__alpha': 0.1}
  R^2 train CV : 0.7072
  R^2 test  CV : 0.6603
  RMSE train CV : 0.6530
  RMSE test  CV : 0.6955
  MAE train CV  : 0.4923
  MAE test  CV  : 0.5293

Lasso :
  Meilleurs paramètres : {'model': Lasso(), 'model__alpha': 0.01}
  R^2 train CV : 0.6926
  R^2 test  CV : 0.6483
  RMSE train CV : 0.6690
  RMSE test  CV : 0.7095
  MAE train CV  : 0.5110
  MAE test  CV  : 0.5409

ElasticNet :
  Meilleurs paramètres : {'model': ElasticNet(), 'model__alpha': 0.01, 'model__l1_ratio': 0.1}
  R^2 train CV : 0.7005
  R^2 test  CV : 0.6557
  RMSE train CV : 0.6604
  RMSE test  CV : 0.7011
  MAE train CV  : 0.4999
  MAE test  CV  : 0.5349


In [23]:
print("4. Première exploration des modèles non-linéaires (GradientBoosting et Random Forest)")
print("-"*50)
# modèles d'arbres décisionnels - non linéaires - POUR VOIR SI ILS SURPASSENT LES MODELES LINEAIRES
# --- Pipeline pour le Gradient Boosting ---
boost_pipe = Pipeline([
    ('prep', preproc),
    ('model', GradientBoostingRegressor(random_state=42))
])

# --- Grille d’hyper-paramètres (préfixe 'model__') ---
param_grid_boost = {
    'model__n_estimators':  [100, 200, 300], # nombre d'arbres déterminés, plus il y a d'arbres plus le modèle est capable de corriger les erreurs résiduelles mais plus long à entraîner
    'model__learning_rate': [0.01, 0.1, 0.2], # taux apprentissage - plus il est grand plus le modèle apprend vite mais risque de sur-apprentissage - 0.1 valeur par défaut - 0.01 plus lent mais plus stable et 0.2 plus rapide
    'model__max_depth':     [3, 5, 7] # profondeur des arbres - 3 classique, 5 à 7 plus de flexibilité mais plus risqué
}

# --- Lancement de la GridSearch ---
gs_boost = GridSearchCV(
    boost_pipe,
    param_grid_boost,
    cv=cv,
    scoring=scoring,
    return_train_score=True,
    n_jobs=-1,
    refit='RMSE'
)
gs_boost.fit(X_train, y_train)
print("→ Meilleurs params GradientBoost :", gs_boost.best_params_)

# Pipeline pour Random Forest
rf_pipe = Pipeline([
    ('prep', preproc),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Grille de recherche (attention au préfixe model__)
param_grid_rf = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth':    [None, 5, 10]
}

# Lancement de la GridSearch
gs_rf = GridSearchCV(
    rf_pipe,
    param_grid_rf,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
    refit='RMSE',
    return_train_score=True
)
gs_rf.fit(X_train, y_train)
print("→ Meilleurs params RF :", gs_rf.best_params_)

# Récupère tous les scores CV pour Gradient Boosting
cv_boost = gs_boost.cv_results_
best_boost_idx = gs_boost.best_index_

print("\n=== Résultats Gradient Boosting ===")
print(f"R^2 train CV : {cv_boost['mean_train_R2'][best_boost_idx]:.4f}")
print(f"R^2 test  CV : {cv_boost['mean_test_R2'][best_boost_idx]:.4f}")
print(f"RMSE train CV : {-cv_boost['mean_train_RMSE'][best_boost_idx]:.4f}")
print(f"RMSE test  CV : {-cv_boost['mean_test_RMSE'][best_boost_idx]:.4f}")
print(f"MAE train CV : {-cv_boost['mean_train_MAE'][best_boost_idx]:.4f}")
print(f"MAE test  CV : {-cv_boost['mean_test_MAE'][best_boost_idx]:.4f}")

# Idem pour Random Forest
cv_rf = gs_rf.cv_results_
best_rf_idx = gs_rf.best_index_

print("\n=== Résultats Random Forest ===")
print(f"R^2 train CV : {cv_rf['mean_train_R2'][best_rf_idx]:.4f}")
print(f"R^2 test  CV : {cv_rf['mean_test_R2'][best_rf_idx]:.4f}")
print(f"RMSE train CV : {-cv_rf['mean_train_RMSE'][best_rf_idx]:.4f}")
print(f"RMSE test  CV : {-cv_rf['mean_test_RMSE'][best_rf_idx]:.4f}")
print(f"MAE train CV : {-cv_rf['mean_train_MAE'][best_rf_idx]:.4f}")
print(f"MAE test  CV : {-cv_rf['mean_test_MAE'][best_rf_idx]:.4f}")

# ANALYSE :
#Le modèle Gradient Boosting est le meilleur des deux.

#Analyse des résultats

#1. Le Gradient Boosting est plus performant
# Il obtient de meilleurs scores sur l'ensemble des métriques de test :

#R² test CV : 0.6361 (contre 0.6229 pour le Random Forest)

#RMSE test CV : 0.7223 (contre 0.7355)

#MAE test CV : 0.5492 (contre 0.5596)

#Cela signifie qu'il est globalement plus précis et commet des erreurs moins importantes que le Random Forest.

#2. Le Gradient Boosting est plus robuste
# Les deux modèles font du surapprentissage (overfitting), ce qui est normal pour des modèles à base d'arbres. 
# Cependant, le Gradient Boosting est mieux maîtrisé :

#Écart R^2 (train/test) pour Gradient Boosting : 0.8765 - 0.6361 = 0.24

#Écart R^2 (train/test) pour Random Forest : 0.9223 - 0.6229 = 0.30

#L'écart plus faible du Gradient Boosting indique qu'il généralise mieux et est plus fiable sur de nouvelles données.




4. Première exploration des modèles non-linéaires (GradientBoosting et Random Forest)
--------------------------------------------------
→ Meilleurs params GradientBoost : {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100}
→ Meilleurs params RF : {'model__max_depth': 10, 'model__n_estimators': 200}

=== Résultats Gradient Boosting ===
R^2 train CV : 0.8765
R^2 test  CV : 0.6361
RMSE train CV : 0.4242
RMSE test  CV : 0.7223
MAE train CV : 0.3241
MAE test  CV : 0.5492

=== Résultats Random Forest ===
R^2 train CV : 0.9223
R^2 test  CV : 0.6229
RMSE train CV : 0.3366
RMSE test  CV : 0.7355
MAE train CV : 0.2664
MAE test  CV : 0.5596


In [24]:

print("\n5. MÉTHODES D'ENSEMBLE")
print("-" * 50)
# Techniques pour combiner plusieurs modèles
# A) BAGGING - Réduit la variance en combinant plusieurs modèles entraînés sur
# différents échantillons des données d'entraînement
print("A) BAGGING (Bootstrap Aggregating)")
bagging_pipe = Pipeline([
    ('prep', preproc),
    ('model', BaggingRegressor(
        n_estimators=50,
        random_state=42,
        n_jobs=-1
    ))
])
cv_bagging = cross_validate(bagging_pipe, X_train, y_train,cv=cv, scoring=scoring)


bagging_pipe.fit(X_train, y_train)
y_pred_train_bagging = bagging_pipe.predict(X_train)
y_pred_test_bagging  = bagging_pipe.predict(X_test)

print(f"R^2 train : {r2_score(y_train, y_pred_train_bagging):.4f}") # gain modeste par rapport aux linéaires (≃0.25), mais écart train/test ≃0.26  
print(f"R^2 test  : {r2_score(y_test,  y_pred_test_bagging):.4f}") # montre un sur‐apprentissage toujours présent.  
print(f"RMSE train : {np.sqrt(mean_squared_error(y_train, y_pred_train_bagging)):.4f}")
print(f"RMSE test  : {np.sqrt(mean_squared_error(y_test,  y_pred_test_bagging)):.4f}")
print(f"MAE train  : {mean_absolute_error(y_train, y_pred_train_bagging):.4f}")
print(f"MAE test   : {mean_absolute_error(y_test,  y_pred_test_bagging):.4f}")

# B) BOOSTING
print("\nB) BOOSTING (Gradient Boosting)")
# Justification : Réduit le biais en combinant séquentiellement des modèles faibles,
# chaque nouveau modèle corrigeant les erreurs du précédent

boosting_pipe = Pipeline([
    ('prep', preproc),
    ('model', HistGradientBoostingRegressor(random_state=42))
])

# Optimisation des hyperparamètres pour le boosting
param_grid_boost = {
    'model__max_iter': [100, 200],
    'model__learning_rate': [0.05, 0.1],
    'model__max_leaf_nodes': [3, 5]
}

gs_boost = GridSearchCV(boosting_pipe, param_grid_boost, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
gs_boost.fit(X_train, y_train)

cv_boost_results = cross_validate(gs_boost.best_estimator_, X_train, y_train, cv=cv, scoring=scoring)

best_boost = gs_boost.best_estimator_
best_boost.fit(X_train, y_train)
y_pred_train_boost = best_boost.predict(X_train)
y_pred_test_boost  = best_boost.predict(X_test)
print(f"R^2 train : {r2_score(y_train, y_pred_train_boost):.4f}") #étonnant - même résultat que bagging 
print(f"R^2 test  : {r2_score(y_test,  y_pred_test_boost):.4f}")
print(f"RMSE train : {np.sqrt(mean_squared_error(y_train, y_pred_train_boost)):.4f}")
print(f"RMSE test  : {np.sqrt(mean_squared_error(y_test,  y_pred_test_boost)):.4f}")
print(f"MAE train  : {mean_absolute_error(y_train, y_pred_train_boost):.4f}")
print(f"MAE test   : {mean_absolute_error(y_test,  y_pred_test_boost):.4f}")

# C) STACKING
print("\nC) STACKING")
# Justification : Combine les prédictions de plusieurs modèles via un meta-learner
# qui apprend comment optimiser la combinaison

# Modèles de base pour le stacking
base_models = [
    ('ridge', gs_reg.best_estimator_['model']),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
    ('gb', gs_boost.best_estimator_['model'])
]

stacking_pipe = Pipeline([
    ('prep', preproc),
    ('model', StackingRegressor(
        estimators=base_models,
        final_estimator=LinearRegression(),
        cv=3,  # CV interne pour éviter l'overfitting
        n_jobs=-1
    ))
])

cv_stacking = cross_validate(stacking_pipe, X_train, y_train, cv=cv, scoring=scoring)
stacking_pipe.fit(X_train, y_train)
y_pred_train_stacking = stacking_pipe.predict(X_train)
y_pred_test_stacking  = stacking_pipe.predict(X_test)
print(f"R^2 train : {r2_score(y_train, y_pred_train_stacking):.4f}") 
print(f"R^2 test  : {r2_score(y_test,  y_pred_test_stacking):.4f}")     
print(f"RMSE train : {np.sqrt(mean_squared_error(y_train, y_pred_train_stacking)):.4f}")
print(f"RMSE test  : {np.sqrt(mean_squared_error(y_test,  y_pred_test_stacking)):.4f}")
print(f"MAE train  : {mean_absolute_error(y_train, y_pred_train_stacking):.4f}")
print(f"MAE test   : {mean_absolute_error(y_test,  y_pred_test_stacking):.4f}")

# ANALYSE :
# Parmi les méthodes d'ensemble, le stacking se révèle être le
# modèle le plus performant et robuste:

#R² test : 0.6765 (le plus élevé)
#RMSE test : 0.7343 (le plus bas, à égalité avec le Bagging)
# MAE test : 0.5734 (le plus bas, à égalité avec le Bagging)

#Écart R² (train/test) pour Stacking : 0.7184 - 0.6706 = 0.0478
#Écart R² (train/test) pour Boosting : 0.6812 - 0.6172 = 0.0640
#Écart R² (train/test) pour Bagging : 0.9465 - 0.6765 = 0.2700

#Le modèle de Bagging, malgré de bons scores en test, souffre d'un surapprentissage très important et est donc moins fiable.



5. MÉTHODES D'ENSEMBLE
--------------------------------------------------
A) BAGGING (Bootstrap Aggregating)
R^2 train : 0.9465
R^2 test  : 0.6765
RMSE train : 0.2795
RMSE test  : 0.7343
MAE train  : 0.2087
MAE test   : 0.5734

B) BOOSTING (Gradient Boosting)
R^2 train : 0.6812
R^2 test  : 0.6172
RMSE train : 0.6819
RMSE test  : 0.7988
MAE train  : 0.5054
MAE test   : 0.6100

C) STACKING
R^2 train : 0.7184
R^2 test  : 0.6706
RMSE train : 0.6409
RMSE test  : 0.7410
MAE train  : 0.4840
MAE test   : 0.5762


In [25]:
print("\n6. COMPARAISON DES MODÈLES (Validation Croisée)")
print("-" * 70)
# evaluer la meilleure performance de modèles
# 1) On rassemble les scores R2, RMSE, MAE calculés en cross-validation pour chaque modèle testé
results_cv = {
    'Baseline (Dummy)': (
        baseline_r2,
        baseline_rmse,
        baseline_mae
    ),
    'Régression Linéaire': (
        cv_linear ['test_R2'].mean(),
        -cv_linear ['test_RMSE'].mean(),
        -cv_linear ['test_MAE'].mean()
    ),
    
    'Meilleur Régularisé': (
        best_reg_r2,      
        best_reg_rmse,
        best_reg_mae
    ),
    'Bagging': (
        cv_bagging['test_R2'].mean(),
        -cv_bagging['test_RMSE'].mean(),
        -cv_bagging['test_MAE'].mean()
    ),
    'Boosting': (
        cv_boost_results['test_R2'].mean(),
        -cv_boost_results['test_RMSE'].mean(),
        -cv_boost_results['test_MAE'].mean()
    ),
    'Stacking': (
        cv_stacking['test_R2'].mean(),
        -cv_stacking['test_RMSE'].mean(),
        -cv_stacking['test_MAE'].mean()
    )
}
# Affichage des résultats CV
print(f"{'Modèle':<20} {'R^2_CV':<8} {'RMSE_CV':<8} {'MAE_CV':<8}")
print("-" * 50)
for name, (r2, rmse, mae) in results_cv.items():
    print(f"{name:<20} {r2:<8.4f} {rmse:<8.4f} {mae:<8.4f}")
# Sélection du meilleur modèle en fonction du plus faible RMSE
best_model_name = min(results_cv.keys(), key=lambda x: results_cv[x][1])
print(f"\nMeilleur modèle (RMSE le plus faible) : {best_model_name}")
print (f'Meilleur modèle de l\'entrainement')

# 2) Évaluation finale (train vs test)
print("\n6b. ÉVALUATION FINALE TRAIN / TEST")
print("-" * 70)
# Ici on refait une évaluation sur train / test pour comparer : R2, RMSE et MAE sur train et test
# On calcule aussi l'amélioration en RMSE par rapport au Dummy sur le test
# et La cohérence entre RMSE CV et RMSE test
print(f"{'Modèle':<20} {'R^2 tr/te':<15} {'RMSE tr/te':<15} {'MAE tr/te':<15}")
print("-" * 70)
# Map des modèles entraînés avec les meilleurs hyperparamètres
models_map = {
    'Baseline (Dummy)': dummy_pipe,
    'Régression Linéaire': linear_pipe,
    'Meilleur Régularisé': gs_reg.best_estimator_,
    'Bagging': bagging_pipe,
    'Boosting': gs_boost.best_estimator_,
    'Stacking': stacking_pipe
}

for name, mdl in models_map.items():
    # Réentraîner sur train complet
    mdl.fit(X_train, y_train)
    y_tr = mdl.predict(X_train)
    y_te = mdl.predict(X_test)
     # Scores train/test
    r2_tr = r2_score(y_train, y_tr)
    r2_te = r2_score(y_test,  y_te)
    rmse_tr = np.sqrt(mean_squared_error(y_train, y_tr))
    rmse_te = np.sqrt(mean_squared_error(y_test,  y_te))
    mae_tr  = mean_absolute_error(y_train, y_tr)
    mae_te  = mean_absolute_error(y_test,  y_te)
    # amélioration vs baseline (Dummy)
    dummy_rmse = np.sqrt(mean_squared_error(y_test, dummy_pipe.predict(X_test)))
    improvement = (dummy_rmse - rmse_te) / dummy_rmse * 100
    # cohérence CV/Test RMSE
    cv_rmse = results_cv[name][1]
    
    #affichage
    print(f"{name:<20} {r2_tr:.3f}/{r2_te:.3f}   "
      f"{rmse_tr:.3f}/{rmse_te:.3f}      "
      f"{mae_tr:.3f}/{mae_te:.3f}      "
      f"{improvement:+.1f}%")
          
    #Gain net par rapport au Dummy valide l'intérêt du modèle, mais CV vs test assez inégal (16 %), 
   



6. COMPARAISON DES MODÈLES (Validation Croisée)
----------------------------------------------------------------------
Modèle               R^2_CV   RMSE_CV  MAE_CV  
--------------------------------------------------
Baseline (Dummy)     -0.0135  1.2054   0.9522  
Régression Linéaire  0.6603   0.6955   0.5292  
Meilleur Régularisé  0.6603   0.6955   0.5293  
Bagging              0.6143   0.7439   0.5669  
Boosting             0.5773   0.7787   0.5826  
Stacking             0.6624   0.6939   0.5303  

Meilleur modèle (RMSE le plus faible) : Stacking
Meilleur modèle de l'entrainement

6b. ÉVALUATION FINALE TRAIN / TEST
----------------------------------------------------------------------
Modèle               R^2 tr/te       RMSE tr/te      MAE tr/te      
----------------------------------------------------------------------
Baseline (Dummy)     0.000/-0.000   1.208/1.291      0.950/0.987      +0.0%
Régression Linéaire  0.704/0.659   0.657/0.754      0.496/0.592      +41.6%
Meilleur R

In [ ]:
#Nous choisissons pour notre cible de consommation d'énergies le modèle stacking pour plusieurs raisons
#Meilleure Performance : Il obtient le meilleur score R² (0.671) et l'erreur RMSE la plus faible (0.741) 
#    sur le jeu de test, le rendant le plus précis.
# 2. Excellente Généralisation : Il affiche un très faible écart entre les scores d'entraînement (0.718) 
#    et de test (0.671), ce qui prouve qu'il n'y a pas de surapprentissage. C'est un modèle robuste.
# 3. Supériorité sur les modèles simples : Il surpasse nettement les modèles linéaires, 
#    ce qui justifie l'utilisation d'une approche plus complexe pour ce problème.

In [29]:
# 1. Définir les modèles de base avec des noms
base_models = [
    ('ridge', Ridge()),
    ('gb', GradientBoostingRegressor(random_state=42))
]

# 2. Définir le pipeline avec un StackingRegressor complet
pipeline_to_improve = Pipeline([
    ('prep', preproc), 
    ('model', StackingRegressor(
        estimators=base_models,
        final_estimator=LinearRegression() # Le méta-modèle
    ))
])

# 3. Définir la grille en ciblant les modèles de base
# On cible les paramètres du Gradient Boosting ('gb') et du Ridge ('ridge')
param_grid = {
    'model__gb__n_estimators': [100, 200],
    'model__gb__max_depth': [3, 5],
    'model__ridge__alpha': [0.1, 1.0, 10.0]
}

# 4. Mettre en place et lancer GridSearchCV (le reste ne change pas)
print("Lancement de l'optimisation des hyperparamètres du Stacking...")
cv = KFold(n_splits=5, shuffle=True, random_state=42)
gs_final = GridSearchCV(
    pipeline_to_improve,
    param_grid,
    cv=cv,
    scoring='neg_root_mean_squared_error', 
    n_jobs=-1
)

gs_final.fit(X_train, y_train)

print("Optimisation terminée.")
print(f"Meilleurs hyperparamètres trouvés : {gs_final.best_params_}")

Lancement de l'optimisation des hyperparamètres du Stacking...
Optimisation terminée.
Meilleurs hyperparamètres trouvés : {'model__gb__max_depth': 3, 'model__gb__n_estimators': 100, 'model__ridge__alpha': 0.1}
